Load Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

Load Data

In [79]:
df = pd.read_excel('food_affordability.xls', sheet_name='Food_afford_cdp_co_region_ca')
df.head(n = 5)


,ind_id,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,...,median_income,affordability_ratio,LL95_affordability_ratio,UL95_affordability_ratio,se_food_afford,rse_food_afford,food_afford_decile,CA_RR_Affordability,ave_fam_size,version
0,757,Food affordability for female-headed household...,2006-2010,1.0,AIAN,CA,6.0,California,NaN,NaN,...,23777.0,0.315779,0.231517,0.400043,0.042991,13.614342,NaN,1.185347,3.34,2013-04-12 04:33:06.235
1,757,Food affordability for female-headed household...,2006-2010,2.0,Asian,CA,6.0,California,NaN,NaN,...,38508.0,0.194980,0.183065,0.206895,0.006079,3.117814,NaN,0.731900,3.34,2013-04-12 04:33:06.235
2,757,Food affordability for female-headed household...,2006-2010,3.0,AfricanAm,CA,6.0,California,NaN,NaN,...,26192.0,0.286664,0.279661,0.293666,0.003573,1.246349,NaN,1.076054,3.34,2013-04-12 04:33:06.235
3,757,Food affordability for female-headed household...,2006-2010,4.0,Latino,CA,6.0,California,NaN,NaN,...,22858.0,0.328475,0.322637,0.334314,0.002979,0.906881,NaN,1.233004,3.34,2013-04-12 04:33:06.235
4,757,Food affordability for female-headed household...,2006-2010,5.0,NHOPI,CA,6.0,California,NaN,NaN,...,36737.0,0.204379,0.173762,0.234997,0.015621,7.643255,NaN,0.767183,3.34,2013-04-12 04:33:06.235


# Feature Engineering

In [80]:
# make factors 
df['region_code'] = df['region_code'].astype('category')
df['race_eth_code'] = df['race_eth_code'].astype('category')
df['geotypevalue'] = df['geotypevalue'].astype('category')
df['county_fips'] = df['county_fips'].astype('category')

Create features with zip code and lat/lon

In [81]:
all_zipcodes = pd.read_csv('zip_code_database.csv')

counties = ['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn',
       'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen',
       'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino',
       'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange',
       'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou', 'Solano', 'Sonoma',
       'Stanislaus', 'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne',
       'Ventura', 'Yolo', 'Yuba']

city_zipcodes = all_zipcodes[all_zipcodes['primary_city'].isin(counties)]
features_to_join = city_zipcodes[['latitude', 'longitude', 'primary_city']]
features_to_join['county_name'] = features_to_join['primary_city']
features_to_join = features_to_join.drop('primary_city', axis = 1)

/var/folders/8f/l9_yqvn54_ldh6kclbdcgvfh0000gn/T/ipykernel_6373/1792318513.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_to_join['county_name'] = features_to_join['primary_city']


In [82]:
df = pd.merge(df, features_to_join, on = 'county_name', how = 'outer')

In [83]:
numeric_predictors = df.select_dtypes(include=['number']).drop('affordability_ratio', axis = 1) # df
numeric_predictors = numeric_predictors.drop(numeric_predictors.filter(regex='^is').columns, axis=1)
numeric_predictors = numeric_predictors.drop(columns = 'CA_RR_Affordability')
num_pred_names = numeric_predictors.columns

categorical_predictors = df.select_dtypes(include=['object', 'category']) # df
dummy_vars = df.filter(regex='^is')  # assuming dummy variables start with "is"
categorical_predictors = pd.concat([categorical_predictors, dummy_vars], axis=1)
cat_pred_names = categorical_predictors.columns

In [84]:
# consistent types for numeric columns
df[num_pred_names] = df[num_pred_names].apply(pd.to_numeric, errors='coerce')

# consistent types for categorical columns
df[cat_pred_names] = df[cat_pred_names].astype(str)


### Final dataframe to csv

In [85]:
df['affordability_ratio'] = df['affordability_ratio'].fillna(df['affordability_ratio'].mean())

In [86]:
df.to_csv('final_data.csv', index = False)

# Supervised Machine Learning Models

In [9]:
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.compose import ColumnTransformer

### Data Preparation

In [87]:
target_variable = df['affordability_ratio'] # Series

X = df.drop(columns = ['affordability_ratio', 'version'])
y = df['affordability_ratio']


In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=307)
X_train = X_train.drop(columns=['CA_RR_Affordability'])
X_test = X_test.drop(columns=['CA_RR_Affordability'])


numeric_transformer = Pipeline(steps=[
    
    ('imputer', SimpleImputer(strategy='mean')),  
    ('poly', PolynomialFeatures(include_bias=False)),
    ('scaler', StandardScaler())                  
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('filtering', SelectPercentile(f_regression, percentile=50))     
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_pred_names),      
        ('cat', categorical_transformer, cat_pred_names)  
    ])



In [107]:
# may need to use samples for testing out some models
X_train_sample = X_train.sample(n=50000, random_state=42)
y_train_sample = y_train.loc[X_train_sample.index]

# scaling response
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
y_train_scaled = scaler.fit_transform(y_train_sample.values.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.values.reshape(-1, 1))


In [108]:
numeric_X_train = X_train_sample.select_dtypes(include=['number'])
corr = numeric_X_train.corrwith(y_train_sample)
print(corr.sort_values(ascending=False))

UL95_affordability_ratio    0.411894
se_food_afford              0.359404
rse_food_afford             0.279099
LL95_affordability_ratio    0.235160
cost_yr                     0.087981
ave_fam_size                0.085187
latitude                    0.030613
longitude                   0.002389
median_income              -0.420331
food_afford_decile         -0.645867
dtype: float64


### KNeighbors Regressor

In [110]:
# sample KNN Regressor

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor(n_neighbors=5, weights='distance'))  
])
pipe.fit(X_train_sample, y_train_sample)
y_test_pred = pipe.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print(test_mse)

0.001171090354606615


### Linear Regression

In [109]:
# sample Linear Regression

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())  
])
pipe.fit(X_train_sample, y_train_scaled)
y_test_pred = pipe.predict(X_test)
test_mse = mean_squared_error(y_test_scaled, y_test_pred)
print(test_mse)


/Users/madisonwozniak/miniconda3/envs/stat486/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.13353550674687673


In [100]:
# evaluate model using cross-validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipe, X_train_sample, y_train_sample, scoring='neg_mean_squared_error', cv=5)
print(f"Average MSE: {-scores.mean()}")

/Users/madisonwozniak/miniconda3/envs/stat486/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:379: RuntimeWarning: invalid value encountered in sqrt
  X_norms = np.sqrt(row_norms(X.T, squared=True) - n_samples * X_means**2)


Average MSE: 0.012752894738311074


### Decision Trees/Random Forests

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='entropy', max_depth=2)
path = clf.cost_complexity_pruning_path(X_train_sample, y_train_sample)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

ccp_alpha_scores = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp_alpha)
    scores = cross_val_score(clf, X_train_sample, y_train_sample, cv=5)
    ccp_alpha_scores.append((ccp_alpha, scores.mean()))

best_ccp_alpha = max(ccp_alpha_scores, key=lambda x: x[1])[0]

tree_best = DecisionTreeClassifier(random_state=42, ccp_alpha=best_ccp_alpha)
tree_best.fit(X_train_sample, y_train_sample)
train_score = tree_best.score(X_train_sample, y_train_sample)
test_score = tree_best.score(X_test, y_test)

print(f"Best ccp_alpha: {best_ccp_alpha}")
print(f"Training accuracy: {train_score}")
print(f"Test accuracy: {test_score}")

Gradient Boosting (XGBoost, LightGBM)

# Deep Learning Model with Hyperparam. Optimization

# Anomaly Detection or Cluster Analysis

# Dimension Reduction 

# SHAP for Feature Importance